In [1]:
import tensorflow as tf
import matplotlib
from time import sleep
from tqdm import tqdm, trange
import numpy as np  
from tqdm import tqdm, trange
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Conv2D,BatchNormalization,Dense,Activation,add,GlobalAveragePooling2D
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [2]:
def preprocess_01():
    (x_train, y_train), (x_test, y_test)=tf.keras.datasets.cifar10.load_data()
    print("y_train_shape:",y_train.shape)
    y_train=y_train.reshape(50000)
    x_train_normalize =x_train/255.0
    x_test_normalize = x_test/255.0
    y_train_onehot=tf.one_hot( y_train,10)
    y_test_onehot=tf.one_hot( y_test,10)
    
    print("y_train_onehot_shape:",y_train_onehot.shape)
    
    #return x_train_normalize,x_test_normalize,y_train_onehot, y_test_onehot
    return x_train_normalize,x_test_normalize,y_train, y_test,y_train_onehot, y_test_onehot


In [3]:
x_train_normal,x_test_normal,y_train,y_test,y_train_onehot,y_text_onehot=preprocess_01()
y_test=y_test.reshape(10000)
resnet=tf.keras.models.load_model("resnet.h5")
#predict_all=resnet.predict(x_train_normal)

y_train_shape: (50000, 1)
y_train_onehot_shape: (50000, 10)


In [7]:
class one_pixel_attack():
    def __init__(self,model,attack_picture,label):
        self.model=model
        self.label=label
        self.attack_picture=attack_picture
        self.high=attack_picture.shape[0]
        self.wide=attack_picture.shape[1]
        self.deep=attack_picture.shape[2]
        self.parent_num=10
        
        self.parent_fig=0
        self.kid_fig=0
        
        self.kid_position=0 #where we focue to change is by parent or not
        self.new_kid=0
        if self.deep!=1:
            print("this pic is colorful")
        else:
            print("this is black white")
    def next_attack_picture(self,attack_picture,label):
        self.attack_picture=attack_picture
        self.high=attack_picture.shape[0]
        self.wide=attack_picture.shape[1]
        self.deep=attack_picture.shape[2]  
        self.label=label
        
        #plt.imshow(self.attack_picture)
        
    def rand_parent(self,parent_num=10):
        self.parent_num=parent_num
        
        high_rand=np.random.randint(low=0,high=self.high,size=parent_num).reshape(parent_num,1)
        wide_rand=np.random.randint(low=0,high=self.wide,size=parent_num).reshape(parent_num,1)
            
        y=np.random.random(parent_num*self.deep).reshape(parent_num,self.deep)
        #y is random RGB the num is farm 0 to 1
            
        parent=np.concatenate((high_rand,wide_rand,y),axis=1)
        self.parent=parent
        #print("the 3D picture attack parent is :")
        #print(self.parent) 
    def find_new_kid(self,mut=0.8):
        
        find_kid=np.random.randint(low=0,high=self.parent_num,size=3)
        while (np.unique(find_kid).shape[0])!=3:
            find_kid=np.random.randint(low=0,high=self.parent_num,size=3)
        #print("we make kids with these parent:")
        #print(find_kid)
        self.kid_position= find_kid[0]
        #print("we well going to find out if the num in :",self.kid_position," is going to change")
        
        
        new=self.parent[find_kid[0]]+mut*(self.parent[find_kid[1]]-self.parent[find_kid[2]])
                
        
        cross_point=np.random.rand((self.deep+2))<0.5 
        
        new_kid=np.where(cross_point,new,self.parent[find_kid[0]])
        
        #clip let all in range
        new_kid[0]=np.clip(new_kid[0],0,self.high-1)
        new_kid[1]=np.clip(new_kid[1],0,self.wide-1)
        
        new_kid[2]=np.clip(new_kid[2],0,1) # num from 0 to 1
        
        new_kid[3]=np.clip(new_kid[3],0,1)
        new_kid[4]=np.clip(new_kid[4],0,1)
          
        self.new_kid=new_kid
        #print("new kid is ",self.new_kid)
        #print("you may calculate fig next")  
    def untarget_attack(self):
        fig=self.attack_picture
        parent=self.parent[self.kid_position]
        kid=self.new_kid
        
        parent_img=fig.copy()
        parent_img[int(parent[0]),int(parent[1]),0]=parent[2]
        parent_img[int(parent[0]),int(parent[1]),1]=parent[3]
        parent_img[int(parent[0]),int(parent[1]),2]=parent[4]
        
        
        kid_img=fig.copy()
        kid_img[int(kid[0]),int(kid[1]),0]=kid[2]
        kid_img[int(kid[0]),int(kid[1]),1]=kid[3]
        kid_img[int(kid[0]),int(kid[1]),2]=kid[4]
        
        #plt.subplot(1,2,1)
        #plt.imshow(parent_img)
        #plt.subplot(1,2,2)
        #plt.imshow(kid_img)
        #plt.show()
        
        parent_result=self.model(parent_img.reshape(1,32,32,3)).numpy().reshape(10)
        kid_result=self.model(kid_img.reshape(1,32,32,3)).numpy().reshape(10)
        #print("parent_result=",parent_result[self.label])
        #print("kid_result=",kid_result[self.label])
        if parent_result[self.label]>kid_result[self.label] :
            #print("change",self.parent[self.kid_position],"to",self.new_kid)
            self.parent[self.kid_position]=self.new_kid
    def attack(self,epoch):
        for a in range(epoch):
            self.find_new_kid()
            
            self.untarget_attack()
    def print_result(self,row):
        fig=plt.gcf()
        fig.set_size_inches(12,14)
        for a in range(self.parent_num):
            parent=self.parent[a]
            
            temp_img=self.attack_picture.copy()
            temp_img[int(parent[0]),int(parent[1]),0]=parent[2]
            temp_img[int(parent[0]),int(parent[1]),1]=parent[3]
            temp_img[int(parent[0]),int(parent[1]),2]=parent[4]
            
            result=self.model(temp_img.reshape(1,32,32,3)).numpy().reshape(10)
            tile="predict rate as "+str(self.label)+"is"+str(result[self.label])
            
            ax=plt.subplot(row,self.parent_num/row,a+1)
            ax.imshow(temp_img,cmap='binary')
            ax.set_title(tile,fontsize=10)
            ax.set_xticks([])
            ax.set_yticks([])
        
            
 
            #output_img[a]=temp_img
        plt.show()
    def success(self,_name):
        for a in range(self.parent_num):
            #print(self.parent[a])
            temp_img=self.attack_picture.copy()
            temp_true=self.attack_picture.copy()
            
            temp_img[int(self.parent[a,0]),int(self.parent[a,1]),0]=self.parent[a,2]
            temp_img[int(self.parent[a,0]),int(self.parent[a,1]),1]=self.parent[a,3]
            temp_img[int(self.parent[a,0]),int(self.parent[a,1]),2]=self.parent[a,4]
            temp_img=temp_img.reshape(1,32,32,3)
            #print(np.argmax(self.model(temp_img)),self.label)
            if np.argmax(self.model(temp_img))!=self.label:
                #matplotlib.image.imsave('one_pixel_attack/num'+str(_name)+'.png', temp_img.reshape(32,32,3))
                #print("noise label",self.model(temp_img))
                #print("original label",self.model(temp_true.reshape(1,32,32,3)))
                #plt.imshow(temp_img.reshape(32,32,3))
                #plt.show()
                #print("true")
                return True
        #print("false")
        return False
            
            
            
        

In [8]:
def attack_ep(ep=100,parent_num=10,num=10):   
    success=0
    fail=0
    attmun=0

    for b in tqdm(range(num)):
        #print(np.argmax(one_pounch.model(x_test_normal[b].reshape(1,32,32,3))))
        #print(y_test[b])
        if np.argmax(one_pounch.model(x_test_normal[b].reshape(1,32,32,3)))==y_test[b]:
            attmun+=1
            one_pounch.next_attack_picture(x_test_normal[b],y_test[b])
            one_pounch.rand_parent(parent_num)
            one_pounch.attack(ep)

            suc=one_pounch.success(b)
            if suc:
                success+=1
            elif not suc:
                fail+=1
            else:
                print("not success error")
    print("success:",success)
    print("fail:",fail)
    
    


In [ ]:
one_pounch=one_pixel_attack(resnet,x_test_normal[0],y_test[0])
attack_ep(ep=1000,parent_num=10,num=10000)

this pic is colorful



  0%|                                                                                        | 0/10000 [00:00<?, ?it/s]
Exception in thread Thread-6:
Traceback (most recent call last):
  File "D:\anaconda\envs\tensorflow\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "D:\anaconda\envs\tensorflow\lib\site-packages\tqdm\_monitor.py", line 62, in run
    for instance in self.tqdm_cls._instances:
  File "D:\anaconda\envs\tensorflow\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

 59%|█████████████████████████████████████████▌                             | 5859/10000 [62:56:23<44:29:03, 38.67s/it]

In [ ]:
# one_pounch=one_pixel_attack(resnet,x_train_normal[0],y_train[0])
# one_pounch.rand_parent(10)
# one_pounch.attack(1000)
# one_pounch.print_result(2)